# AdaptSize
Paper: https://www.usenix.org/conference/nsdi17/technical-sessions/presentation/berger
## Overview
AdaptSize is Admission algorithm that use modified size probabilistic to admit an object. AdaptSize recalculate the C for every N request. 

I've implemented AdaptSize on libCacheSim and below is an example on how to run it.

## Setup Server to Run Our Experiment

In [107]:
from chi import context

# During the transition period, we need to opt into the some of the 
# new python-chi functions. Otherwise the functional interface will
# return the old types.
context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

In [108]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 31 compute_cascadelake_r nodes ready to use


In [109]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-cachesim", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease haekal_muhamat_gmail_com-cachesim has reached status active


In [110]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-cachesim",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server haekal_muhamat_gmail_com-cachesim's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,haekal_muhamat_gmail_com-cachesim
Id,5b4dfa4c-23ef-4b3b-a91a-48af40e4b705
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.0.89 (v4) Type: fixed MAC: bc:97:e1:c4:30:80
Network Name,sharednet1
Created At,2025-04-11T09:26:00Z
Keypair,haekal_muhamat_gmail_com-jupyter
Reservation Id,0cf500b2-3c58-411c-b052-58702a8d728d
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


In [111]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.70 port 22.


Connection successful


## Setup Our Experiment

### Install Dependencies

In [113]:
my_server.execute("sudo apt -qq update && sudo apt install -yqq cmake libzstd-dev libglib2.0-dev")

133 packages can be upgraded. Run 'apt list --upgradable' to see them.


The following additional packages will be installed:
  cmake-data dh-elpa-helper emacsen-common libblkid-dev libffi-dev
  libglib2.0-0 libglib2.0-bin libglib2.0-dev-bin libjsoncpp25 libmount-dev
  libpcre16-3 libpcre2-16-0 libpcre2-32-0 libpcre2-dev libpcre2-posix3
  libpcre3-dev libpcre32-3 libpcrecpp0v5 librhash0 libselinux1-dev
  libsepol-dev pkg-config uuid-dev
Suggested packages:
  cmake-doc ninja-build cmake-format libgirepository1.0-dev libglib2.0-doc
  libgdk-pixbuf2.0-bin | libgdk-pixbuf2.0-dev libxml2-utils
The following NEW packages will be installed:
  cmake cmake-data dh-elpa-helper emacsen-common libblkid-dev libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libjsoncpp25 libmount-dev libpcre16-3
  libpcre2-16-0 libpcre2-32-0 libpcre2-dev libpcre2-posix3 libpcre3-dev
  libpcre32-3 libpcrecpp0v5 librhash0 libselinux1-dev libsepol-dev libzstd-dev
  pkg-config uuid-dev
The following packages will be upgraded:
  libglib2.0-0 libglib2.0-bin
2 upgraded, 24 newly installed, 0 to rem

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Need to get 13.9 MB of archives.
After this operation, 55.9 MB of additional disk space will be used.
(Reading database ... 84249 files and directories currently installed.)
Preparing to unpack .../00-libglib2.0-bin_2.72.4-0ubuntu2.4_amd64.deb ...
Unpacking libglib2.0-bin (2.72.4-0ubuntu2.4) over (2.72.4-0ubuntu2.3) ...
Preparing to unpack .../01-libglib2.0-0_2.72.4-0ubuntu2.4_amd64.deb ...
Unpacking libglib2.0-0:amd64 (2.72.4-0ubuntu2.4) over (2.72.4-0ubuntu2.3) ...
Selecting previously unselected package libjsoncpp25:amd64.
Preparing to unpack .../02-libjsoncpp25_1.9.5-3_amd64.deb ...
Unpacking libjsoncpp25:amd64 (1.9.5-3) ...
Selecting previously unselected package librhash0:amd64.
Preparing to unpack .../03-librhash0_1.4.2-1ubuntu1_amd64.deb ...
Unpacking librhash0:amd64 (1.4.2-1ubuntu1) ...
Selecting previously unselected package dh-elpa-helper.
Preparing to unpack .../04-dh-elpa-helper_2.0.9ubuntu1_all.deb ...
Unpacking dh-elpa-helper (2.0.9ubuntu1) ...
Selecting previously unsel

<Result cmd='sudo apt -qq update && sudo apt install -yqq cmake libzstd-dev libglib2.0-dev' exited=0>

### Clone LibCacheSim Repository

In [114]:
my_server.execute("cd ~ && git clone https://github.com/1a1a11a/libCacheSim")
my_server.execute("ls")
my_server.execute("pwd")

Cloning into 'libCacheSim'...


README
libCacheSim
my_mounting_point
openrc
/home/cc


<Result cmd='pwd' exited=0>

### Build LibCacheSim

In [116]:
my_server.execute("cd ~/libCacheSim && mkdir -p _build && cd _build && cmake .. -Wno-dev && make -j -s && sudo make install")

-- CMAKE_CXX_FLAGS_DEBUG -g CMAKE_CXX_FLAGS_RELWITHDEBINFO -O2 -g -DNDEBUG CMAKE_CXX_FLAGS_RELEASE -O3 -DNDEBUG
-- SUPPORT TTL OFF, USE_HUGEPAGE ON, LOGLEVEL INFO, ENABLE_GLCACHE OFF, ENABLE_LRB OFF, ENABLE_3L_CACHE OFF, OPT_SUPPORT_ZSTD_TRACE ON
-- Found Zstd: /usr/lib/x86_64-linux-gnu/libzstd.so
-- ZSTD_INCLUDE_DIR /usr/include, ZSTD_LIBRARIES /usr/lib/x86_64-linux-gnu/libzstd.so
-- Not Found Tcmalloc: Tcmalloc_LIBRARY-NOTFOUND
-- <<++=====------------------\/------------------=====++>>
-- <<++              libCacheSim summary             ++>>
-- <<++=====------------------/\------------------=====++>>
-- ================== dependency related ==================
-- glib found?     TRUE - LIBS=/usr/lib/x86_64-linux-gnu/libglib-2.0.so, header=/usr/include/glib-2.0;/usr/lib/x86_64-linux-gnu/glib-2.0/include
-- tcmalloc found? FALSE - LIBS=, header=
-- ZSTD found? TRUE - LIBS=/usr/lib/x86_64-linux-gnu/libzstd.so, header=/usr/include
-- ==================== CMake related ==================

<Result cmd='cd ~/libCacheSim && mkdir -p _build && cd _build && cmake .. -Wno-dev && make -j -s && sudo make install' exited=0>

### Test by Running `cachesim --help`

In [117]:
my_server.execute("cachesim --help")

Usage: cachesim [OPTION...] trace_path trace_type eviction_algo cache_size
example: ./cachesim /trace/path csv LRU 100MB

trace can be zstd compressed
cache_size is in byte, but also support KB/MB/GB
supported trace_type: txt/csv/twr/vscsi/oracleGeneralBin
supported eviction_algo: LRU/LFU/FIFO/ARC/LeCaR/Cacheus
print-head-req: Print the first few requests when simulating start

 trace reader related parameters

  -n, --num-req=-1           Num of requests to process, default -1 means all
                             requests in the trace
  -s, --sample-ratio=1       Sample ratio, 1 means no sampling, 0.01 means
                             sample 1% of objects
  -t, --trace-type-params="obj-id-col=1;delimiter=,"
                             Parameters used for csv trace, e.g.,
                             "obj-id-col=1;delimiter=,"

 cache related parameters:

      --admission-params="prob=0.8"
                             params for admission algorithm
  -a, --admission=bloom-filter 

<Result cmd='cachesim --help' exited=0>

## Run The Experiment
You can use included trace, but the trace is small. My recommendation would be to use traces from this link -> https://ftp.pdl.cmu.edu/pub/datasets/twemcacheWorkload/cacheDatasets/

### Function

In [118]:
def lru(p, t, s, opt=''):
    return f"cd ~/libCacheSim/data && cachesim {p} {t} LRU {','.join(str(x) for x in s)} {opt}".strip()

def bloomfilter(p, t, s, opt=''):
    return lru(p,t,s,opt) + ' -a bloom-filter'

def adaptsize(p, t, s, opt=''):
    return lru(p,t,s,opt) + ' -a adaptsize'

We can use LRU for eviction algorithm and use it as baseline

### Use included trace

You can see all included trace here.

In [119]:
my_server.execute("cd ~/libCacheSim/data && ls")

cloudPhysicsIO.csv
cloudPhysicsIO.oracleGeneral.bin
cloudPhysicsIO.txt
cloudPhysicsIO.vscsi
twitter_cluster52.csv
twitter_cluster52_10m.csv.zst


<Result cmd='cd ~/libCacheSim/data && ls' exited=0>

In [120]:
cache_size = [0.1,0.2]
my_server.execute(lru("cloudPhysicsIO.oracleGeneral.bin", "oracleGeneral", cache_size))
my_server.execute(bloomfilter("cloudPhysicsIO.oracleGeneral.bin", "oracleGeneral", cache_size))
my_server.execute(adaptsize("cloudPhysicsIO.oracleGeneral.bin", "oracleGeneral", cache_size))

[INFO]  04-11-2025 09:38:40 cli_reader_utils.c:253  (tid=140262062863488): calculating working set size...
[INFO]  04-11-2025 09:38:40 cli_reader_utils.c:279  (tid=140262062863488): working set size: 48974 object 2029769728 byte
[INFO]  04-11-2025 09:38:40 cli_parser.c:553  (tid=140262062863488): trace path: cloudPhysicsIO.oracleGeneral.bin, trace_type ORACLE_GENERAL_TRACE, ofilepath result/cloudPhysicsIO.oracleGeneral.bin.cachesim, 96 threads, warmup -1 sec, total 1 algo x 2 size = 2 caches, LRU
[INFO]  04-11-2025 09:38:40 simulator.c:288  (tid=140262062863488): simulate_with_multi_caches starts computation, num_warmup_req 0, start cache LRU size 194MiB, end cache LRU size 387MiB, 2 caches, 96 threads, please wait



cloudPhysicsIO.oracleGeneral.bin evict:LRU[] admit:[] cache size      193MiB, 113872 req, miss ratio 0.8097, byte miss ratio 0.9518
cloudPhysicsIO.oracleGeneral.bin evict:LRU[] admit:[] cache size      387MiB, 113872 req, miss ratio 0.7333, byte miss ratio 0.8642


[INFO]  04-11-2025 09:38:44 cli_reader_utils.c:253  (tid=140050625156224): calculating working set size...
[INFO]  04-11-2025 09:38:44 cli_reader_utils.c:279  (tid=140050625156224): working set size: 48974 object 2029769728 byte
[INFO]  04-11-2025 09:38:44 cli_parser.c:553  (tid=140050625156224): trace path: cloudPhysicsIO.oracleGeneral.bin, trace_type ORACLE_GENERAL_TRACE, ofilepath result/cloudPhysicsIO.oracleGeneral.bin.cachesim, 96 threads, warmup -1 sec, total 1 algo x 2 size = 2 caches, LRU, admission: bloom-filter
[INFO]  04-11-2025 09:38:44 simulator.c:288  (tid=140050625156224): simulate_with_multi_caches starts computation, num_warmup_req 0, start cache LRU size 194MiB, end cache LRU size 387MiB, 2 caches, 96 threads, please wait



cloudPhysicsIO.oracleGeneral.bin evict:LRU[] admit:BloomFilter[] cache size      193MiB, 113872 req, miss ratio 0.8142, byte miss ratio 0.9365
cloudPhysicsIO.oracleGeneral.bin evict:LRU[] admit:BloomFilter[] cache size      387MiB, 113872 req, miss ratio 0.7763, byte miss ratio 0.8875


[INFO]  04-11-2025 09:38:48 cli_reader_utils.c:253  (tid=140147849294976): calculating working set size...
[INFO]  04-11-2025 09:38:48 cli_reader_utils.c:279  (tid=140147849294976): working set size: 48974 object 2029769728 byte
[INFO]  04-11-2025 09:38:48 cli_parser.c:553  (tid=140147849294976): trace path: cloudPhysicsIO.oracleGeneral.bin, trace_type ORACLE_GENERAL_TRACE, ofilepath result/cloudPhysicsIO.oracleGeneral.bin.cachesim, 96 threads, warmup -1 sec, total 1 algo x 2 size = 2 caches, LRU, admission: adaptsize
[INFO]  04-11-2025 09:38:48 simulator.c:288  (tid=140147849294976): simulate_with_multi_caches starts computation, num_warmup_req 0, start cache LRU size 194MiB, end cache LRU size 387MiB, 2 caches, 96 threads, please wait



cloudPhysicsIO.oracleGeneral.bin evict:LRU[] admit:AdaptSize[] cache size      193MiB, 113872 req, miss ratio 0.7134, byte miss ratio 0.8988
cloudPhysicsIO.oracleGeneral.bin evict:LRU[] admit:AdaptSize[] cache size      387MiB, 113872 req, miss ratio 0.6830, byte miss ratio 0.8661


<Result cmd='cd ~/libCacheSim/data && cachesim cloudPhysicsIO.oracleGeneral.bin oracleGeneral LRU 0.1,0.2 -a adaptsize' exited=0>

### Use bigger trace

Define the urls, you can find it here https://ftp.pdl.cmu.edu/pub/datasets/twemcacheWorkload/cacheDatasets/

In [121]:
url = [
    "https://ftp.pdl.cmu.edu/pub/datasets/twemcacheWorkload/cacheDatasets/twitter/cluster50.oracleGeneral.zst",
    "https://ftp.pdl.cmu.edu/pub/datasets/twemcacheWorkload/cacheDatasets/twitter/cluster53.oracleGeneral.zst"
]

Download it

In [122]:
def download(u):
    print("Downloading", u)
    my_server.execute(f"cd ~/libCacheSim/data && wget -q {u}")

for u in url:
    download(u)

Our experiment function

In [123]:
def run_exp(cache_size):
    trace = [ u[u.rfind("/") + 1:] for u in url ]
    for t in trace:
        my_server.execute(lru(t, "oracleGeneral", cache_size))
        my_server.execute(bloomfilter(t, "oracleGeneral", cache_size))
        my_server.execute(adaptsize(t, "oracleGeneral", cache_size, "--admission-params=\"max-iteration=15,reconf-interval=3000000\""))

Our experiment, we use relative cache size

In [124]:
run_exp([0.01, 0.1])

[INFO]  04-11-2025 09:39:39 cli_reader_utils.c:253  (tid=140082643137664): calculating working set size...
[INFO]  04-11-2025 09:39:49 cli_reader_utils.c:276  (tid=140082643137664): estimated working set size (0.09 sample ratio): 2660988 object 43347397808 byte
[INFO]  04-11-2025 09:39:49 cli_parser.c:553  (tid=140082643137664): trace path: cluster50.oracleGeneral.zst, trace_type ORACLE_GENERAL_TRACE, ofilepath result/cluster50.oracleGeneral.zst.cachesim, 96 threads, warmup -1 sec, total 1 algo x 2 size = 2 caches, LRU
[INFO]  04-11-2025 09:39:49 simulator.c:288  (tid=140082643137664): simulate_with_multi_caches starts computation, num_warmup_req 0, start cache LRU size 413MiB, end cache LRU size 4GiB, 2 caches, 96 threads, please wait



cluster50.oracleGeneral.zst evict:LRU[] admit:[] cache size      413MiB, 139655585 req, miss ratio 0.6013, byte miss ratio 0.7479
cluster50.oracleGeneral.zst evict:LRU[] admit:[] cache size     4133MiB, 139655585 req, miss ratio 0.2122, byte miss ratio 0.2266


[INFO]  04-11-2025 09:40:21 cli_reader_utils.c:253  (tid=139751536112768): calculating working set size...
[INFO]  04-11-2025 09:40:31 cli_reader_utils.c:276  (tid=139751536112768): estimated working set size (0.09 sample ratio): 2660988 object 43347397808 byte
[INFO]  04-11-2025 09:40:31 cli_parser.c:553  (tid=139751536112768): trace path: cluster50.oracleGeneral.zst, trace_type ORACLE_GENERAL_TRACE, ofilepath result/cluster50.oracleGeneral.zst.cachesim, 96 threads, warmup -1 sec, total 1 algo x 2 size = 2 caches, LRU, admission: bloom-filter
[INFO]  04-11-2025 09:40:31 simulator.c:288  (tid=139751536112768): simulate_with_multi_caches starts computation, num_warmup_req 0, start cache LRU size 413MiB, end cache LRU size 4GiB, 2 caches, 96 threads, please wait



cluster50.oracleGeneral.zst evict:LRU[] admit:BloomFilter[] cache size      413MiB, 139655585 req, miss ratio 0.6108, byte miss ratio 0.7464
cluster50.oracleGeneral.zst evict:LRU[] admit:BloomFilter[] cache size     4133MiB, 139655585 req, miss ratio 0.2222, byte miss ratio 0.2282


[INFO]  04-11-2025 09:41:21 cli_reader_utils.c:253  (tid=140026206577792): calculating working set size...
[INFO]  04-11-2025 09:41:31 cli_reader_utils.c:276  (tid=140026206577792): estimated working set size (0.09 sample ratio): 2660988 object 43347397808 byte
[INFO]  04-11-2025 09:41:31 cli_parser.c:553  (tid=140026206577792): trace path: cluster50.oracleGeneral.zst, trace_type ORACLE_GENERAL_TRACE, ofilepath result/cluster50.oracleGeneral.zst.cachesim, 96 threads, warmup -1 sec, total 1 algo x 2 size = 2 caches, LRU, admission: adaptsize, admission-params: max-iteration=15,reconf-interval=3000000
[INFO]  04-11-2025 09:41:31 simulator.c:288  (tid=140026206577792): simulate_with_multi_caches starts computation, num_warmup_req 0, start cache LRU size 413MiB, end cache LRU size 4GiB, 2 caches, 96 threads, please wait


...


cluster53.oracleGeneral.zst evict:LRU[] admit:[] cache size      103MiB, 246508262 req, miss ratio 0.3147, byte miss ratio 0.3374
cluster53.oracleGeneral.zst evict:LRU[] admit:[] cache size     1033MiB, 246508262 req, miss ratio 0.1088, byte miss ratio 0.0954


[INFO]  04-11-2025 09:48:22 cli_reader_utils.c:253  (tid=139743772447872): calculating working set size...
[INFO]  04-11-2025 09:48:39 cli_reader_utils.c:276  (tid=139743772447872): estimated working set size (0.09 sample ratio): 7243918 object 10832632250 byte
[INFO]  04-11-2025 09:48:39 cli_parser.c:553  (tid=139743772447872): trace path: cluster53.oracleGeneral.zst, trace_type ORACLE_GENERAL_TRACE, ofilepath result/cluster53.oracleGeneral.zst.cachesim, 96 threads, warmup -1 sec, total 1 algo x 2 size = 2 caches, LRU, admission: bloom-filter
[INFO]  04-11-2025 09:48:39 simulator.c:288  (tid=139743772447872): simulate_with_multi_caches starts computation, num_warmup_req 0, start cache LRU size 103MiB, end cache LRU size 1GiB, 2 caches, 96 threads, please wait



cluster53.oracleGeneral.zst evict:LRU[] admit:BloomFilter[] cache size      103MiB, 246508262 req, miss ratio 0.3363, byte miss ratio 0.3386
cluster53.oracleGeneral.zst evict:LRU[] admit:BloomFilter[] cache size     1033MiB, 246508262 req, miss ratio 0.1315, byte miss ratio 0.0974


[INFO]  04-11-2025 09:49:31 cli_reader_utils.c:253  (tid=139875206001792): calculating working set size...
[INFO]  04-11-2025 09:49:48 cli_reader_utils.c:276  (tid=139875206001792): estimated working set size (0.09 sample ratio): 7243918 object 10832632250 byte
[INFO]  04-11-2025 09:49:48 cli_parser.c:553  (tid=139875206001792): trace path: cluster53.oracleGeneral.zst, trace_type ORACLE_GENERAL_TRACE, ofilepath result/cluster53.oracleGeneral.zst.cachesim, 96 threads, warmup -1 sec, total 1 algo x 2 size = 2 caches, LRU, admission: adaptsize, admission-params: max-iteration=15,reconf-interval=3000000
[INFO]  04-11-2025 09:49:48 simulator.c:288  (tid=139875206001792): simulate_with_multi_caches starts computation, num_warmup_req 0, start cache LRU size 103MiB, end cache LRU size 1GiB, 2 caches, 96 threads, please wait



cluster53.oracleGeneral.zst evict:LRU[] admit:AdaptSize[max-iteration=15,reconf-interval=3000000] cache size      103MiB, 246508262 req, miss ratio 0.2345, byte miss ratio 0.5614
cluster53.oracleGeneral.zst evict:LRU[] admit:AdaptSize[max-iteration=15,reconf-interval=3000000] cache size     1033MiB, 246508262 req, miss ratio 0.1051, byte miss ratio 0.1802
